In [1]:
import collections
import os
import sys
import time

import cv2
import numpy as np
from IPython import display
from openvino.inference_engine import IECore

sys.path.append("../utils")
import notebook_utils as utils

In [5]:
# directory where model will be downloaded
base_model_dir = "F:/python_workspace/openvino/model"

# model name as named in Open Model Zoo
model_name = "ssdlite_mobilenet_v2"

model_downloader = "'E:/Program Files (x86)/Intel/openvino_2021.4.752/deployment_tools/open_model_zoo/tools/downloader/downloader.py'"

precision = "FP16"

# output path for the conversion
converted_model_path = f"F:/python_workspace/openvino/model/public/{model_name}/{precision}/{model_name}.xml"

In [10]:


# download_command = f"{model_downloader} " \
#                    f"--name {model_name} " \
#                    f"--output_dir {base_model_dir} " \
#                    f"--cache_dir {base_model_dir}"
# ! $download_command

''E:' is not recognized as an internal or external command,
operable program or batch file.


In [17]:
%cd "E:\Program Files (x86)\Intel\openvino_2021.4.752\deployment_tools\open_model_zoo\tools\downloader"
!python downloader.py --name {model_name} --output_dir {base_model_dir} --cache_dir {base_model_dir}

E:\Program Files (x86)\Intel\openvino_2021.4.752\deployment_tools\open_model_zoo\tools\downloader
################|| Downloading ssdlite_mobilenet_v2 ||################

========== Downloading F:\python_workspace\openvino\model\public\ssdlite_mobilenet_v2\ssdlite_mobilenet_v2_coco_2018_05_09.tar.gz
... 2%, 1024 KB, 819 KB/s, 1 seconds passed
... 4%, 2048 KB, 845 KB/s, 2 seconds passed
... 6%, 3072 KB, 949 KB/s, 3 seconds passed
... 8%, 4096 KB, 1048 KB/s, 3 seconds passed
... 10%, 5120 KB, 1145 KB/s, 4 seconds passed
... 12%, 6144 KB, 1228 KB/s, 5 seconds passed
... 14%, 7168 KB, 1295 KB/s, 5 seconds passed
... 16%, 8192 KB, 1358 KB/s, 6 seconds passed
... 18%, 9216 KB, 1417 KB/s, 6 seconds passed
... 20%, 10240 KB, 1462 KB/s, 7 seconds passed
... 22%, 11264 KB, 1505 KB/s, 7 seconds passed
... 24%, 12288 KB, 1545 KB/s, 7 seconds passed
... 26%, 13312 KB, 1566 KB/s, 8 seconds passed
... 28%, 14336 KB, 1590 KB/s, 9 seconds passed
... 30%, 15360 KB, 1624 KB/s, 9 seconds passed
... 32%, 16

In [18]:
precision = "FP16"

# output path for the conversion
converted_model_path = f"F:/python_workspace/openvino/model/public/{model_name}/{precision}/{model_name}.xml"

# if not os.path.exists(converted_model_path):
#     convert_command = f"omz_converter " \
#                       f"--name {model_name} " \
#                       f"--download_dir {base_model_dir} " \
#                       f"--precisions {precision}"
#     ! $convert_command

'omz_converter' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
%cd "E:\Program Files (x86)\Intel\openvino_2021.4.752\deployment_tools\open_model_zoo\tools\downloader"

E:\Program Files (x86)\Intel\openvino_2021.4.752\deployment_tools\open_model_zoo\tools\downloader


In [6]:
!python converter.py --name {model_name} --download_dir {base_model_dir} --precisions {precision}

========== Converting ssdlite_mobilenet_v2 to IR (FP16)
Conversion command: C:\ProgramData\Anaconda3\python.exe -m mo --framework=tf --data_type=FP16 --output_dir=F:\python_workspace\openvino\model\public\ssdlite_mobilenet_v2\FP16 --model_name=ssdlite_mobilenet_v2 --reverse_input_channels --input_shape=[1,300,300,3] --input=image_tensor --output=detection_scores,detection_boxes,num_detections "--transformations_config=E:\Program Files (x86)\Intel\openvino_2021.4.752\deployment_tools\model_optimizer/extensions/front/tf/ssd_v2_support.json" --tensorflow_object_detection_api_pipeline_config=F:\python_workspace\openvino\model\public\ssdlite_mobilenet_v2/ssdlite_mobilenet_v2_coco_2018_05_09/pipeline.config --input_model=F:\python_workspace\openvino\model\public\ssdlite_mobilenet_v2/ssdlite_mobilenet_v2_coco_2018_05_09/frozen_inference_graph.pb

Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	F:\python_workspace\openvino\model\public\ssdlite_mobilenet_v2/ssdlite_mo

2021-12-29 22:31:21.765801: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library cudart64_110.dll
C:\Users\nkele\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\autograph\impl\api.py:22: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp


In [7]:
# initialize inference engine
ie_core = IECore()
# read the network and corresponding weights from file
net = ie_core.read_network(model=converted_model_path)
# load the model on the CPU (you can choose manually CPU, GPU, MYRIAD etc.)
# or let the engine choose best available device (AUTO)
exec_net = ie_core.load_network(network=net, device_name="CPU")

# get input and output names of nodes
input_key = list(exec_net.input_info)[0]
output_key = list(exec_net.outputs.keys())[0]

# get input size
height, width = exec_net.input_info[input_key].tensor_desc.dims[2:]

In [8]:
input_key, output_key

('image_tensor', 'DetectionOutput')

In [9]:
# https://tech.amikelive.com/node-718/what-object-categories-labels-are-in-coco-dataset/
classes = [
    "background", "person", "bicycle", "car", "motorcycle", "airplane", "bus", "train",
    "truck", "boat", "traffic light", "fire hydrant", "street sign", "stop sign",
    "parking meter", "bench", "bird", "cat", "dog", "horse", "sheep", "cow", "elephant",
    "bear", "zebra", "giraffe", "hat", "backpack", "umbrella", "shoe", "eye glasses",
    "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite",
    "baseball bat", "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle",
    "plate", "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana", "apple",
    "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut", "cake", "chair",
    "couch", "potted plant", "bed", "mirror", "dining table", "window", "desk", "toilet",
    "door", "tv", "laptop", "mouse", "remote", "keyboard", "cell phone", "microwave", "oven",
    "toaster", "sink", "refrigerator", "blender", "book", "clock", "vase", "scissors",
    "teddy bear", "hair drier", "toothbrush", "hair brush"
]


# colors for above classes (Rainbow Color Map)
colors = cv2.applyColorMap(
    src=np.arange(0, 255, 255 / len(classes), dtype=np.float32).astype(np.uint8),
    colormap=cv2.COLORMAP_RAINBOW
).squeeze()


def process_results(frame, results, thresh=0.6):
    # size of the original frame
    h, w = frame.shape[:2]
    # results is a tensor [1, 1, 100, 7]
    results = results[output_key][0][0]
    boxes = []
    labels = []
    scores = []
    for _, label, score, xmin, ymin, xmax, ymax in results:
        # create a box with pixels coordinates from the box with normalized coordinates [0,1]
        boxes.append(tuple(map(int, (xmin * w, ymin * h, xmax * w, ymax * h))))
        labels.append(int(label))
        scores.append(float(score))

    # apply non-maximum suppression to get rid of many overlapping entities
    # see https://paperswithcode.com/method/non-maximum-suppression
    # this algorithm returns indices of objects to keep
    indices = cv2.dnn.NMSBoxes(bboxes=boxes, scores=scores, score_threshold=thresh, nms_threshold=0.6)

    # if there are no boxes
    if len(indices) == 0:
        return []

    # filter detected objects
    return [(labels[idx], scores[idx], boxes[idx]) for idx in indices.flatten()]


def draw_boxes(frame, boxes):
    for label, score, box in boxes:
        # choose color for the label
        color = tuple(map(int, colors[label]))
        # draw box
        cv2.rectangle(img=frame, pt1=box[:2], pt2=box[2:], color=color, thickness=3)
        # draw label name inside the box
        cv2.putText(img=frame, text=f"{classes[label]}", org=(box[0] + 10, box[1] + 30),
                    fontFace=cv2.FONT_HERSHEY_COMPLEX, fontScale=frame.shape[1] / 1000, color=color,
                    thickness=1, lineType=cv2.LINE_AA)

    return frame

In [10]:
# main processing function to run object detection
def run_object_detection(source=0, flip=False, use_popup=False):
    # create video player to play with target fps
    player = utils.VideoPlayer(source=source, flip=flip, fps=30)
    # start capturing
    player.start()
    try:
        if use_popup:
            title = "Press ESC to Exit"
            cv2.namedWindow(winname=title, flags=cv2.WINDOW_GUI_NORMAL | cv2.WINDOW_AUTOSIZE)

        processing_times = collections.deque()
        while True:
            # grab the frame
            frame = player.next()
            if frame is None:
                print("Source ended")
                break
            # if frame larger than full HD, reduce size to improve the performance
            scale = 1280 / max(frame.shape)
            if scale < 1:
                frame = cv2.resize(src=frame, dsize=None, fx=scale, fy=scale,
                                   interpolation=cv2.INTER_AREA)

            # resize image and change dims to fit neural network input
            input_img = cv2.resize(src=frame, dsize=(width, height), interpolation=cv2.INTER_AREA)
            # create batch of images (size = 1)
            input_img = input_img.transpose(2, 0, 1)[np.newaxis, ...]

            # measure processing time
            start_time = time.time()
            # get results
            results = exec_net.infer(inputs={input_key: input_img})
            stop_time = time.time()
            # get poses from network results
            boxes = process_results(frame=frame, results=results)

            # draw boxes on a frame
            frame = draw_boxes(frame=frame, boxes=boxes)

            processing_times.append(stop_time - start_time)
            # use processing times from last 200 frames
            if len(processing_times) > 200:
                processing_times.popleft()

            _, f_width = frame.shape[:2]
            # mean processing time [ms]
            processing_time = np.mean(processing_times) * 1000
            fps = 1000 / processing_time
            cv2.putText(img=frame, text=f"Inference time: {processing_time:.1f}ms ({fps:.1f} FPS)", org=(20, 40),
                        fontFace=cv2.FONT_HERSHEY_COMPLEX, fontScale=f_width / 1000,
                        color=(0, 0, 255), thickness=1, lineType=cv2.LINE_AA)

            # use this workaround if there is flickering
            if use_popup:
                cv2.imshow(winname=title, mat=frame)
                key = cv2.waitKey(1)
                # escape = 27
                if key == 27:
                    break
            else:
                # encode numpy array to jpg
                _, encoded_img = cv2.imencode(ext=".jpg", img=frame,
                                              params=[cv2.IMWRITE_JPEG_QUALITY, 100])
                # create IPython image
                i = display.Image(data=encoded_img)
                # display the image in this notebook
                display.clear_output(wait=True)
                display.display(i)
    # ctrl-c
    except KeyboardInterrupt:
        print("Interrupted")
    # any different error
    except RuntimeError as e:
        print(e)
    finally:
        # stop capturing
        player.stop()
        if use_popup:
            cv2.destroyAllWindows()

In [13]:
video_file = "F:/python_workspace/openvino/bottle-detection.mp4"

run_object_detection(source=video_file, flip=False, use_popup=True)